# Example of an LSTM network for sentiment analysis

ref: https://github.com/dragen1860/TensorFlow-2.x-Tutorials/blob/master/09-RNN-Sentiment-Analysis/main.py

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [3]:

np.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 10000
# truncate and pad input sequences
max_review_length = 80
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=top_words)
# X_train = tf.convert_to_tensor(X_train)
# y_train = tf.one_hot(y_train, depth=2)
print('Pad sequences (samples x time)')
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_review_length)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

17465344/17464789 [==============================] - 7s 0us/step
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [8]:
X_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [12]:
y_train[0]

1

# LSTM Model

In [4]:
class RNN(keras.Model):

    def __init__(self, units, num_classes, num_layers):
        super(RNN, self).__init__()

        
        # have 1000 words totally, every word will be embedding into 100 length vector
        # the max sentence lenght is 80 words
        self.embedding = keras.layers.Embedding(top_words, 100, input_length=max_review_length)

        self.rnn = keras.layers.LSTM(units, return_sequences=True)
        self.rnn2 = keras.layers.LSTM(units)

        self.fc = keras.layers.Dense(1)

    def call(self, inputs, training=None, mask=None):

        x = self.embedding(inputs)
        x = self.rnn(x) 
        x = self.rnn2(x) 

        x = self.fc(x)
        print(x.shape)

        return x

# Train Model

In [13]:
model.summary()

Model: "rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  42240     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
embedding (Embedding)        multiple                  1000000   
_________________________________________________________________
dense (Dense)                multiple                  65        
Total params: 1,075,329
Trainable params: 1,075,329
Non-trainable params: 0
_________________________________________________________________


In [5]:
units = 64
num_classes = 2
batch_size = 32
epochs = 20

model = RNN(units, num_classes, num_layers=2)


model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test), verbose=1)

# evaluate on test set
scores = model.evaluate(x_test, y_test, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

(None, 1)
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
(None, 1)
(None, 1)
25000/25000 [==============================] - 35s 1ms/sample - loss: 0.4209 - accuracy: 0.7932 - val_loss: 0.3735 - val_accuracy: 0.8202
Epoch 2/20
25000/25000 [==============================] - 28s 1ms/sample - loss: 0.2774 - accuracy: 0.8816 - val_loss: 0.3649 - val_accuracy: 0.8379
Epoch 3/20
25000/25000 [==============================] - 27s 1ms/sample - loss: 0.1949 - accuracy: 0.9213 - val_loss: 0.4539 - val_accuracy: 0.8292
Epoch 4/20
25000/25000 [==============================] - 27s 1ms/sample - loss: 0.1317 - accuracy: 0.9478 - val_loss: 0.4900 - val_accuracy: 0.8292
Epoch 5/20
25000/25000 [==============================] - 27s 1ms/sample - loss: 0.0899 - accuracy: 0.9676 - val_loss: 0.6378 - val_accuracy: 0.8054
Epoch 6/20
25000/25000 [==============================] - 27s 1ms/sample - loss: 0.0695 - accuracy: 0.9762 - val_loss: 0.6513 - val_accuracy: 0.8174
Epoch 7/20
25000/25000 [==